In [3]:
import pandas as pd
import numpy as np
import ast

# Load dataset
file_path = "/Users/ashokkumarsinha/Downloads/TRADES.csv"  # Update with actual path
df = pd.read_csv(file_path)

# Data Cleaning and Preprocessing
df['Trade_History'] = df['Trade_History'].astype(str)
df['Trade_History'] = df['Trade_History'].apply(lambda x: '[]' if x in ['nan', 'None', ''] else x)
df['Trade_History'] = df['Trade_History'].apply(ast.literal_eval)
df = df.explode('Trade_History')
df = df.dropna(subset=['Trade_History'])
df = df.join(pd.json_normalize(df['Trade_History']))
df = df.drop(columns=['Trade_History'])

# Convert timestamp and clean data
df['timestamp'] = pd.to_datetime(df['time'], unit='ms')
df.drop(columns=['time'], inplace=True)
df.dropna(inplace=True)

# Classify positions
df['position_type'] = df['side'] + '_' + df['positionSide']

# Calculate Metrics per Account
def calculate_account_metrics(group):
    metrics = {}
    
    # Profit and Loss
    metrics['PnL'] = group['realizedProfit'].sum()
    
    # ROI Calculation
    total_investment = group['quantity'].sum()
    metrics['ROI'] = (metrics['PnL'] / total_investment) * 100 if total_investment != 0 else 0
    
    # Win Rate
    win_trades = group[group['realizedProfit'] > 0]
    metrics['Win Rate'] = (len(win_trades) / len(group)) * 100
    
    # Win Positions
    metrics['Win Positions'] = len(win_trades)
    metrics['Total Positions'] = len(group)
    
    # Daily Returns
    group['daily_returns'] = group['price'].pct_change()
    
    # Sharpe Ratio (Risk-free rate assumed 0)
    returns = group['daily_returns']
    if len(returns) > 1:
        metrics['Sharpe Ratio'] = returns.mean() / (returns.std() + 1e-6)
    else:
        metrics['Sharpe Ratio'] = 0
    
    # Maximum Drawdown
    def calculate_mdd(profits):
        cumulative_profits = profits.cumsum()
        max_so_far = cumulative_profits.cummax()
        drawdown = (cumulative_profits - max_so_far) / max_so_far
        return drawdown.min() if len(drawdown) > 0 else 0
    
    metrics['MDD'] = calculate_mdd(group['realizedProfit'])
    
    return pd.Series(metrics)

# Group by Port_IDs and calculate metrics
account_metrics = df.groupby('Port_IDs').apply(calculate_account_metrics).reset_index()

# Normalize Metrics (Min-Max Scaling)
def min_max_normalize(series):
    min_val = series.min()
    max_val = series.max()
    return (series - min_val) / (max_val - min_val) if max_val != min_val else series

metrics_to_normalize = ['ROI', 'PnL', 'Win Rate', 'Sharpe Ratio']
for metric in metrics_to_normalize:
    account_metrics[f'Normalized_{metric}'] = min_max_normalize(account_metrics[metric])

# Improved Final Score Calculation with Positive Bias
account_metrics['Final Score'] = (
    account_metrics['Normalized_ROI'] * 0.3 +        # 30% weight
    account_metrics['Normalized_PnL'] * 0.25 +       # 25% weight
    account_metrics['Normalized_Win Rate'] * 0.2 +   # 20% weight
    account_metrics['Normalized_Sharpe Ratio'] * 0.15 +  # 15% weight
    (1 - min_max_normalize(account_metrics['MDD'])) * 0.1  # 10% weight (inverse of MDD)
)

# Rank Accounts
df_ranked = account_metrics.sort_values(by='Final Score', ascending=False)
df_top_20 = df_ranked.head(20)

# Save Results
df_ranked.to_csv("/Users/ashokkumarsinha/Downloads/ranked_accounts.csv", index=False)
df_top_20.to_csv("/Users/ashokkumarsinha/Downloads/top_20_accounts.csv", index=False)

# Print Results
print("Top 20 Accounts:")
print(df_top_20[['Port_IDs', 'Final Score', 'ROI', 'PnL', 'Win Rate', 'Sharpe Ratio']])
print("\nRanking Complete. Top 20 Accounts saved.")

Top 20 Accounts:
                Port_IDs  Final Score       ROI            PnL  Win Rate  \
15   3887577207880438784     0.849696  3.845048  169088.642497     100.0   
28   3932103299427844097     0.688015  3.845048   59735.195497     100.0   
78   4000877324693233921     0.669009  3.848953   46674.638252     100.0   
98   4021669203289716224     0.627746  3.464756   39020.070699     100.0   
68   3993014919980212480     0.624347  3.848953   16467.436391     100.0   
52   3966142151544441601     0.621710  3.845048   14889.298496     100.0   
83   4008537296438699777     0.611563  3.841143    8232.373929     100.0   
128  4033614723417828608     0.610575  3.841143    7564.237454     100.0   
48   3956076827719377409     0.610202  3.464756   27153.902549     100.0   
97   4021243448368889856     0.607291  3.848953    4931.227103     100.0   
57   3977116548751698176     0.607180  3.841143    5268.219947     100.0   
67   3991414786174551297     0.606779  3.845048    4790.608179     100.